In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import sqlite3
import mysql.connector as cnt
import time


In [2]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butai/puslapis/1/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [44]:
bs = BeautifulSoup(source, 'html.parser')

butai_nuorodos = []

linkai = bs.find_all('h3')
for linkas in linkai:
    # print('::REM::')
    l = linkas.find('a')
    if l != None:
        # print('error')
        # print(l['href'])
        butai_nuorodos.append(l['href'])
        
print(len(butai_nuorodos))
print(butai_nuorodos)
  

25
['https://www.aruodas.lt/butai-vilniuje-zirmunuose-riteriu-g-tik-ka-naujai-irengtas-kambariu-butas-1-3472873/', 'https://www.aruodas.lt/butai-palangoje-palangoje-a-mickeviciaus-g-norime-pristatyti-jums-musu-parduodamus-1-3427260/', 'https://www.aruodas.lt/butai-vilniuje-pasilaiciuose-perkunkiemio-g-brokeriu-paslaugos-nedomina-1-3411527/', 'https://www.aruodas.lt/butai-palangoje-palangoje-medvalakio-g-parduodamas-dvieju-kambariu-butas-naujame-1-3481853/', 'https://www.aruodas.lt/butai-palangoje-palangoje-medvalakio-g-parduodamas-butas-naujame-kvartale-su-sildomu-1-3485145/', 'https://www.aruodas.lt/butai-kaune-silainiuose-v-kavolio-g-butai-tiesiogiai-is-statytojo-silainiuose-1-3480325/', 'https://www.aruodas.lt/butai-kaune-sanciuose-a-juozapaviciaus-pr-butas-yra-isskirtiniame-wwwjuozapaviciaus-1-3342907/', 'https://www.aruodas.lt/butai-kaune-vilijampoleje-raudondvario-pl-palankiomis-salygomis-suteikiame-kredita-1-3414581/', 'https://www.aruodas.lt/butai-vilniuje-rasose-stepono-batoro

In [45]:
kitas = bs.find('div', {'class':'pagination'}).find_all('a', {'class':'page-bt'})
for k in kitas:
    # print(k.text.strip())
    if k.text.strip() == '»':
        kitas_psl = k['href']
        # print(kitas_psl)
next_nuoroda = f'https://www.aruodas.lt/{kitas_psl}'

print(next_nuoroda)

https://www.aruodas.lt//butai/puslapis/2/?FOwnerDbId0=1&FOwnerDbId2=1


In [56]:
driver = webdriver.Chrome(options=opcijos)
for nuoroda in butai_nuorodos[0:1]:
    print(nuoroda)
    
    
    # url = nuoroda
    # url = 'https://www.aruodas.lt/butai-palangoje-palangoje-medvalakio-g-parduodamas-dvieju-kambariu-butas-naujame-1-3481853/'
    # url = 'https://www.aruodas.lt/butai-palangoje-palangoje-a-mickeviciaus-g-norime-pristatyti-jums-musu-parduodamus-1-3427260/'
    url = 'https://www.aruodas.lt/butai-palangoje-palangoje-a-mickeviciaus-g-norime-pristatyti-jums-musu-parduodamus-1-3427260/'
    driver.get(url)
    time.sleep(2)
    source = driver.page_source
    
driver.close()

https://www.aruodas.lt/butai-vilniuje-zirmunuose-riteriu-g-tik-ka-naujai-irengtas-kambariu-butas-1-3472873/


In [58]:
# driver = webdriver.Chrome(options=opcijos)
# for nuoroda in butai_nuorodos[0:1]:
# print(nuoroda)

duomenys = {
    'miestas':'None',
    'rajonas':'None',
    'gatve':'None',
    'kaina':'None',
    'kv_kaina':'None',
    'Plotas:':'None',
    'Kambarių sk.:':'None',
    'Metai:':'None',
    'Aukštas:':'None',
    'Aukštų sk.:':'None',
    'Namo numeris:':'None',
    'Buto numeris:':'None',
    'Pastato tipas:':'None',
    'Šildymas:':'None',
    'Įrengimas:':'None',
    'Pastato energijos suvartojimo klasė:':'None',
    'Ypatybės:':'None',
    'Papildomos patalpos:':'None',
    'Papildoma įranga:':'None',
    'Apsauga:':'None',
    'nuoroda':'None'
}

# duomenys['nuoroda'] = nuoroda

# url = nuoroda
# url = 'https://www.aruodas.lt/butai-palangoje-palangoje-medvalakio-g-parduodamas-dvieju-kambariu-butas-naujame-1-3481853/'
# driver.get(url)
# time.sleep(2)
# source = driver.page_source

bs_buto = BeautifulSoup(source, 'html.parser')

gyvenviete = bs_buto.find('h1', {'class':'obj-header-text'}).text
gyvenviete_lst = gyvenviete.split(',')
if len(gyvenviete_lst) == 4:
    miestas = gyvenviete_lst[0].strip()
    duomenys['miestas'] = miestas
    # print(miestas)
    rajonas = gyvenviete_lst[1].strip()
    duomenys['rajonas'] = rajonas
    # print(rajonas)
    gatve = gyvenviete_lst[2].strip()
    duomenys['gatve'] = gatve
    # print(gatve)
if len(gyvenviete_lst) == 3:
    miestas = gyvenviete_lst[0].strip()
    duomenys['miestas'] = miestas
    # print(miestas)
    gatve = gyvenviete_lst[1].strip()
    duomenys['gatve'] = gatve

    
kaina = int(bs_buto.find('span', {'class':'price-eur'}).text.strip().replace('€', '').replace(' ', ''))
duomenys['kaina'] = kaina
# print(kaina)

kv_kaina_txt = bs_buto.find('span', {'class':'price-per'}).text.strip()
# print(kv_kaina_txt)
kv_kaina = int(kv_kaina_txt.split('€')[0].replace(' ', '').replace('(', ''))
duomenys['kv_kaina'] = kv_kaina
# print(kv_kaina)


info_keys = []
info_values = []
info_k = bs_buto.find('dl', {'class':'obj-details'}).find_all('dt')
for i in info_k:
    # print(i.text.strip())
    info_keys.append(i.text.strip())
    a = i.text.strip()
    if a not in duomenys.keys():
        print(a)
print(info_keys)
print(len(info_keys))

# info_v = bs_buto.find('dl', {'class':'obj-details'}).find_all('span',{'class':'fieldValueContainer'})
info_v = bs_buto.find('dl', {'class':'obj-details'}).find_all('dd')
for i in info_v:
    # print(i.text.strip())
    info_values.append(i.text.strip())
print(info_values)
print(len(info_values))

for k, v in zip(info_keys, info_values):
    duomenys[k] = v

print(duomenys)

data = tuple(duomenys.values())
data1 = []
data1.append(data)
    
    # SDB = sqlite3.connect('Aruodas_test.db')  # jei neegzistuoja db , bus sukurta nauja db
    # Cs = SDB.cursor()
    
    # #  if not exists - tikrina ar jau sukurta DB
    # sql = '''create table if not exists Aruodas
    # (
    # miestas text not null,
    # rajonas text not null,
    # gatve text not null,
    # kaina numeric not null,
    # kvadatoKaina numeric not null,
    # plotas text not null,
    # kambariai text not null,
    # metai text not null,
    # aukstas text not null,
    # aukstuSk text not null,
    # namoNr text not null,
    # butoNr text not null,
    # pastatoTipas text not null,
    # sildymas text not null,
    # irengimas text not null,
    # energetineKlase text not null,
    # ypatybes text not null,
    # papildomosPatalpos text not null,
    # papildomaIranga text not null,
    # apsauga text not null,
    # nuoroda text not null
    # )
    # '''
    # Cs.execute(sql)

    # sql_template = '''insert into Aruodas values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
    # Cs.executemany(sql_template, data1)

    # SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

    # SDB.close()
    # print('data written to DB')     
    
    
    
# driver.close()
print('Baigta!!!')

# .text
# .contents
# .content

Ypatybės:
['Plotas:', 'Kambarių sk.:', 'Aukštas:', 'Aukštų sk.:', 'Metai:', 'Pastato tipas:', 'Šildymas:', 'Įrengimas:', 'Pastato energijos suvartojimo klasė:', 'Ypatybės:', 'Papildomos patalpos:', 'Papildoma įranga:', 'Apsauga:']
13
['35 m²', '2', '1', '3', '2023', 'Kita', 'Centrinis kolektorinis', 'Įrengtas', 'B', 'Nauja kanalizacija\nNauja elektros instaliacija\nInternetas', 'Sandėliukas\nTerasa\nVieta automobiliui', 'Kondicionierius\nSu baldais\nŠaldytuvas\nŠildomos grindys\nVirtuvės komplektas\nViryklė\nPlastikiniai vamzdžiai\nIndaplovė\nDušo kabina', 'Šarvuotos durys\nSignalizacija\nKodinė laiptinės spyna\nVaizdo kameros']
13
{'miestas': 'Palanga', 'rajonas': 'Palanga', 'gatve': 'A. Mickevičiaus g.', 'kaina': 268000, 'kv_kaina': 7657, 'Plotas:': '35 m²', 'Kambarių sk.:': '2', 'Metai:': '2023', 'Aukštas:': '1', 'Aukštų sk.:': '3', 'Namo numeris:': 'None', 'Buto numeris:': 'None', 'Pastato tipas:': 'Kita', 'Šildymas:': 'Centrinis kolektorinis', 'Įrengimas:': 'Įrengtas', 'Pastato en

In [ ]:
# dėl laiko apribojimo

# t = now()
# now() - t
# DeltaTimeStamp
# Timedelta
# .total_seconds

# https://www.geeksforgeeks.org/python-difference-between-two-dates-in-minutes-using-datetime-timedelta-method/

In [3]:
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.aruodas.lt/butai/puslapis/1/?FOwnerDbId0=1&FOwnerDbId2=1'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [4]:
bs = BeautifulSoup(source, 'html.parser')

cities = []

miestai = bs.find('div', {'id':'searchFormField_FRegion'}).find_all('label', {'class':'dropDownLabel'})
for miestas in miestai:
    i  = miestas.text.strip()
    if i != '':
        # print(i)
        cities.append(i)
        
print(cities)
print(len(cities))

cities_for_link = ['vilniuje', 'kaune', 'klaipedoje', 'siauliuose', 'panevezyje', 'alytuje', 'palangoje',
                   'akmeneje', 'alytaus-rajone', 'anysciuose', 'birstone', 'birzuose', 'druskininkuose',
                   'elektrenuose', 'ignalinoje', 'jonavoje', 'joniskyje', 'jurbarke', 'kaisiadoryse',
                   'kalvarijoje', 'kauno-rajone', 'kazlu-rudoje', 'kedainiuose', 'kelmeje', 'klaipedos-rajone',
                   'kretingoje', 'kupiskyje', 'lazdijuose', 'marijampoleje', 'mazeikiuose', 'moletuose',
                   'neringoje', 'pagegiuose', 'pakruojyje', 'panevezio-rajone', 'pasvalyje', 'plungeje',
                   'prienuose', 'radvilisyje', 'raseiniuose', 'rietave', 'rokiskyje', 'skuode', 'sakiuose',
                   'salcininkuose', 'siauliu-rajone', 'silaleje', 'siluteje', 'sirvintuose', 'svencionyse',
                   'taurageje', 'telsiuose', 'trakuose', 'ukmergeje', 'utenoje', 'varenoje', 'vilkaviskyje',
                   'vilniaus-rajone', 'visagine', 'zarasuose']

print(cities_for_link)
print(len(cities_for_link))
    

['Vilnius', 'Kaunas', 'Klaipėda', 'Šiauliai', 'Panevėžys', 'Alytus', 'Palanga', 'Akmenės r.', 'Alytaus r.', 'Anykščių r.', 'Birštono sav.', 'Biržų r.', 'Druskininkų sav.', 'Elektrėnų sav.', 'Ignalinos r.', 'Jonavos r.', 'Joniškio r.', 'Jurbarko r.', 'Kaišiadorių r.', 'Kalvarijos sav.', 'Kauno r.', 'Kazlų Rūdos sav.', 'Kėdainių r.', 'Kelmės r.', 'Klaipėdos r.', 'Kretingos r.', 'Kupiškio r.', 'Lazdijų r.', 'Marijampolės sav.', 'Mažeikių r.', 'Molėtų r.', 'Neringos sav.', 'Pagėgių sav.', 'Pakruojo r.', 'Panevėžio r.', 'Pasvalio r.', 'Plungės r.', 'Prienų r.', 'Radviliškio r.', 'Raseinių r.', 'Rietavo sav.', 'Rokiškio r.', 'Skuodo r.', 'Šakių r.', 'Šalčininkų r.', 'Šiaulių r.', 'Šilalės r.', 'Šilutės r.', 'Širvintų r.', 'Švenčionių r.', 'Tauragės r.', 'Telšių r.', 'Trakų r.', 'Ukmergės r.', 'Utenos r.', 'Varėnos r.', 'Vilkaviškio r.', 'Vilniaus r.', 'Visagino sav.', 'Zarasų r.']
60
['vilniuje', 'kaune', 'klaipedoje', 'siauliuose', 'panevezyje', 'alytuje', 'palangoje', 'akmeneje', 'alytaus-

In [5]:
for idx, city in enumerate(cities):
    print(idx, city)

0 Vilnius
1 Kaunas
2 Klaipėda
3 Šiauliai
4 Panevėžys
5 Alytus
6 Palanga
7 Akmenės r.
8 Alytaus r.
9 Anykščių r.
10 Birštono sav.
11 Biržų r.
12 Druskininkų sav.
13 Elektrėnų sav.
14 Ignalinos r.
15 Jonavos r.
16 Joniškio r.
17 Jurbarko r.
18 Kaišiadorių r.
19 Kalvarijos sav.
20 Kauno r.
21 Kazlų Rūdos sav.
22 Kėdainių r.
23 Kelmės r.
24 Klaipėdos r.
25 Kretingos r.
26 Kupiškio r.
27 Lazdijų r.
28 Marijampolės sav.
29 Mažeikių r.
30 Molėtų r.
31 Neringos sav.
32 Pagėgių sav.
33 Pakruojo r.
34 Panevėžio r.
35 Pasvalio r.
36 Plungės r.
37 Prienų r.
38 Radviliškio r.
39 Raseinių r.
40 Rietavo sav.
41 Rokiškio r.
42 Skuodo r.
43 Šakių r.
44 Šalčininkų r.
45 Šiaulių r.
46 Šilalės r.
47 Šilutės r.
48 Širvintų r.
49 Švenčionių r.
50 Tauragės r.
51 Telšių r.
52 Trakų r.
53 Ukmergės r.
54 Utenos r.
55 Varėnos r.
56 Vilkaviškio r.
57 Vilniaus r.
58 Visagino sav.
59 Zarasų r.


In [10]:
pasirink = input('Įvesk mieto numerius, atskirtus kableliais, iš kurių nori gauti skelbimus: ')
pasirink_pvz = '8,23,35' # Alytus, Kelme, Pasvalys
pas = [int(i) for i in pasirink.split(',')]
pasirinktas_miestas_link = []
for i in pas:
    pasirinktas_miestas_link.append(cities_for_link[i])
print(pasirinktas_miestas_link)


['panevezyje', 'kalvarijoje', 'pakruojyje', 'alytuje']
